In [1]:
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'


import findspark
findspark.init()
findspark.find()

'/usr/lib/spark/'

In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("yarn") \
        .config("spark.driver.cores", "2") \
        .config("spark.driver.memory", "2g") \
        .appName("myapp2") \
        .getOrCreate()

print(spark)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
2023-08-09 20:22:02,446 WARN util.Utils: Your hostname, fhm351qam71cljcmr1am resolves to a loopback address: 127.0.1.1; using 172.16.0.55 instead (on interface eth0)
2023-08-09 20:22:02,447 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-08-09 20:22:04,375 WARN util.Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update 

In [26]:
# events = spark.read.json("/user/master/data/events/date=2022-05-31")
events = spark.read.json("/user/master/data/events")

In [31]:
events.printSchema()

root
 |-- event: struct (nullable = true)
 |    |-- admins: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- channel_id: long (nullable = true)
 |    |-- datetime: string (nullable = true)
 |    |-- media: struct (nullable = true)
 |    |    |-- media_type: string (nullable = true)
 |    |    |-- src: string (nullable = true)
 |    |-- message: string (nullable = true)
 |    |-- message_channel_to: long (nullable = true)
 |    |-- message_from: long (nullable = true)
 |    |-- message_group: long (nullable = true)
 |    |-- message_id: long (nullable = true)
 |    |-- message_to: long (nullable = true)
 |    |-- message_ts: string (nullable = true)
 |    |-- reaction_from: string (nullable = true)
 |    |-- reaction_type: string (nullable = true)
 |    |-- subscription_channel: long (nullable = true)
 |    |-- subscription_user: string (nullable = true)
 |    |-- tags: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |   

In [32]:
events.write \
        .partitionBy("date","event_type") \
        .mode("overwrite") \
        .parquet("/user/inthemidde/data/events")

KeyboardInterrupt: 

In [30]:
events.describe().show(2)

KeyboardInterrupt: 

In [9]:
ev = spark.read.parquet("/user/inthemidde/data/events")
ev.show(10)

+--------------------+------------+
|               event|  event_type|
+--------------------+------------+
|[,, 2022-05-31 06...|subscription|
|[,, 2022-05-31 07...|subscription|
|[,, 2022-05-31 00...|subscription|
|[,, 2022-05-31 16...|subscription|
|[,, 2022-05-31 00...|subscription|
|[,, 2022-05-31 04...|subscription|
|[,, 2022-05-31 01...|subscription|
|[,, 2022-05-31 02...|subscription|
|[,, 2022-05-31 06...|subscription|
|[,, 2022-05-31 15...|subscription|
+--------------------+------------+
only showing top 10 rows



In [25]:
import pyspark.sql.functions as F
from pyspark.sql.functions import lit
ev = ev.withColumn('date',lit('2022-05-31'))
ev.select('event','date','event_type').orderBy(['event.datetime'], ascending=[0]).show(10)

+--------------------+----------+------------+
|               event|      date|  event_type|
+--------------------+----------+------------+
|[[19342], 987160,...|2022-05-31|     message|
|[,, 2022-05-31 23...|2022-05-31|subscription|
|[[26358], 247511,...|2022-05-31|     message|
|[[79792], 748847,...|2022-05-31|     message|
|[,, 2022-05-31 23...|2022-05-31|subscription|
|[,, 2022-05-31 23...|2022-05-31|subscription|
|[[151897], 396845...|2022-05-31|     message|
|[,, 2022-05-31 23...|2022-05-31|subscription|
|[,, 2022-05-31 23...|2022-05-31|subscription|
|[,, 2022-05-31 23...|2022-05-31|subscription|
+--------------------+----------+------------+
only showing top 10 rows

